# Amazon SageMaker Multi-Model Endpoints using Scikit Learn

*이 노트북은 [Amazon SageMaker Multi-Model Endpoints using Scikit Learn (영문 원본)](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/multi_model_sklearn_home_value/sklearn_multi_model_endpoint_home_value.ipynb) 의 한국어 번역입니다.*

고객들은 [Amazon SageMaker 멀티 모델 엔드포인트(multi-model endpoints)](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html)를 사용하여 최대 수천 개의 모델을 완벽하게 호스팅하는 엔드포인트를 생성할 수 있습니다. 이러한 엔드포인트는 공통 추론 컨테이너(common inference container)에서 제공할 수 있는 많은 모델 중 하나를 온디맨드(on demand)로 호출할 수 있어야 하고 자주 호출되지 않는 모델이 약간의 추가 대기 시간(latency) 허용이 가능한 사례들에 적합합니다. 지속적으로 낮은 추론 대기 시간이 필요한 애플리케이션의 경우 기존의 엔드포인트가 여전히 최선의 선택입니다.

High level에서 Amazon SageMaker는 필요에 따라 멀티 모델 엔드포인트에 대한 모델 로딩 및 언로딩을 관리합니다. 특정 모델에 대한 호출 요청이 발생하면 Amazon SageMaker는 해당 모델에 할당된 인스턴스로 요청을 라우팅하고 S3에서 모델 아티팩트(model artifacts)를 해당 인스턴스로 다운로드한 다음 컨테이너의 메모리에 모델 로드를 시작합니다. 로딩이 완료되면 Amazon SageMaker는 요청된 호출을 수행하고 결과를 반환합니다. 모델이 선택된 인스턴스의 메모리에 이미 로드되어 있으면 다운로드 및 로딩 단계들을 건너 뛰고 즉시 호출이 수행됩니다.

멀티 모델 엔드포인트 작성 및 사용 방법을 보여주기 위해, 이 노트북은 단일 위치의 주택 가격을 예측하는 Scikit Learn 모델을 사용하는 예시를 제공합니다. 이 도메인은 멀티 모델 엔드포인트를 쉽게 실험하기 위한 간단한 예제입니다.

Amazon SageMaker 멀티 모델 엔드포인트 기능은 컨테이너를 가져 오는 프레임워크를 포함한 모든 머신 러닝 프레임워크 및 알고리즘에서 작동하도록 설계되었습니다.

### Contents

1. [Build and register a Scikit Learn container that can serve multiple models](#Build-and-register-a-Scikit-Learn-container-that-can-serve-multiple-models)
1. [Generate synthetic data for housing models](#Generate-synthetic-data-for-housing-models)
1. [Train multiple house value prediction models](#Train-multiple-house-value-prediction-models)
1. [Import models into hosting](#Import-models-into-hosting)
  1. [Deploy model artifacts to be found by the endpoint](#Deploy-model-artifacts-to-be-found-by-the-endpoint)
  1. [Create the Amazon SageMaker model entity](#Create-the-Amazon-SageMaker-model-entity)
  1. [Create the multi-model endpoint](#Create-the-multi-model-endpoint)
1. [Exercise the multi-model endpoint](#Exercise-the-multi-model-endpoint)
  1. [Dynamically deploy another model](#Dynamically-deploy-another-model)
  1. [Invoke the newly deployed model](#Invoke-the-newly-deployed-model)
  1. [Updating a model](#Updating-a-model)
1. [Clean up](#Clean-up)

## Build and register a Scikit Learn container that can serve multiple models

In [1]:
!pip install -qU awscli boto3 sagemaker

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


추론 컨테이너가 멀티 모델 엔드 포인트에서 여러 모델을 제공하려면 특정 모델의 로드(load), 나열(list), 가져오기(get), 언로드(unload) 및 호출(invoke)을 위한 [추가 API](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html)를 구현해야 합니다.

[SageMaker Scikit Learn 컨테이너 저장소의 'mme' branch](https://github.com/aws/sagemaker-scikit-learn-container/tree/mme)는 멀티 모델 엔드포인트에 필요한 추가 컨테이너 API를 구현하는 HTTP 프론트엔드를 제공하는 프레임워크인 [Multi Model Server](https://github.com/awslabs/multi-model-server)를 사용하도록 SageMaker의 Scikit Learn 프레임워크 컨테이너를 조정하는 방법에 대한 예제 구현입니다. 또한 사용자 정의 프레임워크 (본 예시에서는 Scikit Learn 프레임워크)를 사용하여 모델을 제공하기 위한 플러그 가능한 백엔드 핸들러(pluggable backend handler)를 제공합니다.

이 branch를 사용하여 모든 멀티 모델 엔드 포인트 컨테이너 요구 사항을 충족하는 Scikit Learn 컨테이너를 구축한 다음 해당 이미지를 Amazon Elastic Container Registry(ECR)에 업로드합니다. 이미지를 ECR에 업로드하면 새로운 ECR 저장소가 생성될 수 있으므로 이 노트북에는 일반 `SageMakerFullAccess` 권한 외에 권한이 필요합니다. 이러한 권한을 추가하는 가장 쉬운 방법은 관리형 정책 `AmazonEC2ContainerRegistryFullAccess`를 노트북 인스턴스를 시작하는 데 사용한 역할(role)에 추가하는 것입니다. 이 작업을 수행할 때 노트북 인스턴스를 다시 시작할 필요가 없으며 새 권한을 즉시 사용할 수 있습니다.

In [2]:
ALGORITHM_NAME = 'multi-model-sklearn'

In [ ]:
%%sh -s $ALGORITHM_NAME

algorithm_name=$1

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)

ecr_image="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email --registry-ids ${account})

# Build the docker image locally with the image name and then push it to ECR
# with the full image name.

# First clear out any prior version of the cloned repo
rm -rf sagemaker-scikit-learn-container/

# Clone the sklearn container repo
git clone --single-branch --branch mme https://github.com/aws/sagemaker-scikit-learn-container.git
cd sagemaker-scikit-learn-container/

# Build the "base" container image that encompasses the installation of the
# scikit-learn framework and all of the dependencies needed.
docker build -q -t sklearn-base:0.20-2-cpu-py3 -f docker/0.20-2/base/Dockerfile.cpu --build-arg py_version=3 .

# Create the SageMaker Scikit-learn Container Python package.
python setup.py bdist_wheel --universal

# Build the "final" container image that encompasses the installation of the
# code that implements the SageMaker multi-model container requirements.
docker build -q -t ${algorithm_name} -f docker/0.20-2/final/Dockerfile.cpu .

docker tag ${algorithm_name} ${ecr_image}

docker push ${ecr_image}

## Generate synthetic data for housing models

In [ ]:
import numpy as np
import pandas as pd
import json
import datetime
import time
from time import gmtime, strftime
import matplotlib.pyplot as plt

In [ ]:
NUM_HOUSES_PER_LOCATION = 1000
LOCATIONS  = ['NewYork_NY',    'LosAngeles_CA',   'Chicago_IL',    'Houston_TX',   'Dallas_TX',
              'Phoenix_AZ',    'Philadelphia_PA', 'SanAntonio_TX', 'SanDiego_CA',  'SanFrancisco_CA']
PARALLEL_TRAINING_JOBS = 4 # len(LOCATIONS) if your account limits can handle it
MAX_YEAR = 2019

In [ ]:
def gen_price(house):
    _base_price = int(house['SQUARE_FEET'] * 150)
    _price = int(_base_price + (10000 * house['NUM_BEDROOMS']) + \
                               (15000 * house['NUM_BATHROOMS']) + \
                               (15000 * house['LOT_ACRES']) + \
                               (15000 * house['GARAGE_SPACES']) - \
                               (5000 * (MAX_YEAR - house['YEAR_BUILT'])))
    return _price

In [ ]:
def gen_random_house():
    _house = {'SQUARE_FEET':   int(np.random.normal(3000, 750)),
              'NUM_BEDROOMS':  np.random.randint(2, 7),
              'NUM_BATHROOMS': np.random.randint(2, 7) / 2,
              'LOT_ACRES':     round(np.random.normal(1.0, 0.25), 2),
              'GARAGE_SPACES': np.random.randint(0, 4),
              'YEAR_BUILT':    min(MAX_YEAR, int(np.random.normal(1995, 10)))}
    _price = gen_price(_house)
    return [_price, _house['YEAR_BUILT'],   _house['SQUARE_FEET'], 
                    _house['NUM_BEDROOMS'], _house['NUM_BATHROOMS'], 
                    _house['LOT_ACRES'],    _house['GARAGE_SPACES']]

In [ ]:
COLUMNS = ['PRICE', 'YEAR_BUILT', 'SQUARE_FEET', 'NUM_BEDROOMS',
           'NUM_BATHROOMS', 'LOT_ACRES', 'GARAGE_SPACES']
def gen_houses(num_houses):
    _house_list = []
    for i in range(num_houses):
        _house_list.append(gen_random_house())
    _df = pd.DataFrame(_house_list, 
                       columns=COLUMNS)
    return _df

## Train multiple house value prediction models

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
import boto3

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

s3 = boto3.resource('s3')
s3_client = boto3.client('s3')

sagemaker_session = sagemaker.Session()
role = get_execution_role()

ACCOUNT_ID  = boto3.client('sts').get_caller_identity()['Account']
REGION      = boto3.Session().region_name
BUCKET      = sagemaker_session.default_bucket()
SCRIPT_FILENAME     = 'script.py'
USER_CODE_ARTIFACTS = 'user_code.tar.gz'

MULTI_MODEL_SKLEARN_IMAGE = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(ACCOUNT_ID, REGION, 
                                                                           ALGORITHM_NAME)

DATA_PREFIX            = 'DEMO_MME_SCIKIT'
HOUSING_MODEL_NAME     = 'housing'
MULTI_MODEL_ARTIFACTS  = 'multi_model_artifacts'

TRAIN_INSTANCE_TYPE    = 'ml.m4.xlarge'
ENDPOINT_INSTANCE_TYPE = 'ml.m4.xlarge'

### Split a given dataset into train, validation, and test

In [ ]:
from sklearn.model_selection import train_test_split
SEED = 7
SPLIT_RATIOS = [0.6, 0.3, 0.1]

def split_data(df):
    # split data into train and test sets
    seed      = SEED
    val_size  = SPLIT_RATIOS[1]
    test_size = SPLIT_RATIOS[2]
    
    num_samples = df.shape[0]
    X1 = df.values[:num_samples, 1:] # keep only the features, skip the target, all rows
    Y1 = df.values[:num_samples, :1] # keep only the target, all rows

    # Use split ratios to divide up into train/val/test
    X_train, X_val, y_train, y_val = \
        train_test_split(X1, Y1, test_size=(test_size + val_size), random_state=seed)
    # Of the remaining non-training samples, give proper ratio to validation and to test
    X_test, X_test, y_test, y_test = \
        train_test_split(X_val, y_val, test_size=(test_size / (test_size + val_size)), 
                         random_state=seed)
    # reassemble the datasets with target in first column and features after that
    _train = np.concatenate([y_train, X_train], axis=1)
    _val   = np.concatenate([y_val,   X_val],   axis=1)
    _test  = np.concatenate([y_test,  X_test],  axis=1)

    return _train, _val, _test

### Launch a single training job for a given housing location
모델 학습 시, 기존 SageMaker 모델과 동일한 방식으로 학습하기 때문에 멀티 모델 엔트 포인트에 특화된 기능을 따로 구현하실 필요가 없습니다.

In [ ]:
%%writefile $SCRIPT_FILENAME

import argparse
import os
import glob

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.externals import joblib

# inference functions ---------------
def model_fn(model_dir):
    print('loading model.joblib from: {}'.format(model_dir))
    _loaded_model = joblib.load(os.path.join(model_dir, 'model.joblib'))
    return _loaded_model


if __name__ =='__main__':

    print('extracting arguments')
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument('--n-estimators', type=int, default=10)
    parser.add_argument('--min-samples-leaf', type=int, default=3)

    # Data, model, and output directories
    parser.add_argument('--model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--model-name', type=str)

    args, _ = parser.parse_known_args()

    print('reading data')
    print('model_name: {}'.format(args.model_name))

    train_file = os.path.join(args.train, args.model_name + '_train.csv')    
    train_df = pd.read_csv(train_file)

    val_file = os.path.join(args.validation, args.model_name + '_val.csv')
    test_df = pd.read_csv(os.path.join(val_file))

    print('building training and testing datasets')
    X_train = train_df[train_df.columns[1:train_df.shape[1]]] 
    X_test = test_df[test_df.columns[1:test_df.shape[1]]]
    y_train = train_df[train_df.columns[0]]
    y_test = test_df[test_df.columns[0]]

    # train
    print('training model')
    model = RandomForestRegressor(
        n_estimators=args.n_estimators,
        min_samples_leaf=args.min_samples_leaf,
        n_jobs=-1)
    
    model.fit(X_train, y_train)

    # print abs error
    print('validating model')
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print('AE-at-' + str(q) + 'th-percentile: '
              + str(np.percentile(a=abs_err, q=q)))
        
    # persist model
    path = os.path.join(args.model_dir, 'model.joblib')
    joblib.dump(model, path)
    print('model persisted at ' + path)

In [ ]:
#  can test the model locally 
# ! python script.py --n-estimators 100 \
#                    --min-samples-leaf 2 \
#                    --model-dir ./ \
#                    --model-name 'NewYork_NY' \
#                    --train ./data/NewYork_NY/train/ \
#                    --validation ./data/NewYork_NY/val/
# from sklearn.externals import joblib
# regr = joblib.load('./model.joblib')
# _start_time = time.time()
# regr.predict([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
# _duration = time.time() - _start_time
# print('took {:,d} ms'.format(int(_duration * 1000)))

In [ ]:
from sagemaker.sklearn.estimator import SKLearn

def launch_training_job(location):
    # clear out old versions of the data
    _s3_bucket = s3.Bucket(BUCKET)
    _full_input_prefix = '{}/model_prep/{}'.format(DATA_PREFIX, location)
    _s3_bucket.objects.filter(Prefix=_full_input_prefix + '/').delete()

    # upload the entire set of data for all three channels
    _local_folder = 'data/{}'.format(location)
    _inputs = sagemaker_session.upload_data(path=_local_folder, 
                                            key_prefix=_full_input_prefix)
    print('Training data uploaded: {}'.format(_inputs))
    
    _job = 'mme-{}'.format(location.replace('_', '-'))
    _full_output_prefix = '{}/model_artifacts/{}'.format(DATA_PREFIX, 
                                                        location)
    _s3_output_path = 's3://{}/{}'.format(BUCKET, _full_output_prefix)

    _estimator = SKLearn(
         entry_point=SCRIPT_FILENAME, role=role,
         train_instance_count=1, train_instance_type=TRAIN_INSTANCE_TYPE,
         framework_version='0.20.0',
         output_path=_s3_output_path,
         base_job_name=_job,
         metric_definitions=[
             {'Name' : 'median-AE',
              'Regex': 'AE-at-50th-percentile: ([0-9.]+).*$'}],
         hyperparameters = {'n-estimators'    : 100,
                            'min-samples-leaf': 3,
                            'model-name'      : location})
    
    DISTRIBUTION_MODE = 'FullyReplicated'
    _train_input = sagemaker.s3_input(s3_data=_inputs+'/train', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    _val_input   = sagemaker.s3_input(s3_data=_inputs+'/val', 
                                      distribution=DISTRIBUTION_MODE, content_type='csv')
    _remote_inputs = {'train': _train_input, 'validation': _val_input}

    _estimator.fit(_remote_inputs, wait=False)
    
    return _estimator.latest_training_job.name

### Kick off a model training job for each housing location

In [ ]:
def save_data_locally(location, train, val, test):
    _header = ','.join(COLUMNS)
    
    os.makedirs('data/{}/train'.format(location))
    np.savetxt( 'data/{0}/train/{0}_train.csv'.format(location), train, delimiter=',', fmt='%.2f')
    
    os.makedirs('data/{}/val'.format(location))
    np.savetxt( 'data/{0}/val/{0}_val.csv'.format(location),     val,   delimiter=',', fmt='%.2f')
    
    os.makedirs('data/{}/test'.format(location))
    np.savetxt( 'data/{0}/test/{0}_test.csv'.format(location),   test,  delimiter=',', fmt='%.2f')

In [ ]:
import shutil
import os

training_jobs = []

shutil.rmtree('data', ignore_errors=True)

for loc in LOCATIONS[:PARALLEL_TRAINING_JOBS]:
    _houses = gen_houses(NUM_HOUSES_PER_LOCATION)
    _train, _val, _test = split_data(_houses)
    save_data_locally(loc, _train, _val, _test)
    _job = launch_training_job(loc)
    training_jobs.append(_job)
print('{} training jobs launched: {}'.format(len(training_jobs), training_jobs))

### Wait for all model training to finish

In [ ]:
def wait_for_training_job_to_complete(job_name):
    print('Waiting for job {} to complete...'.format(job_name))
    _resp   = sm_client.describe_training_job(TrainingJobName=job_name)
    _status = _resp['TrainingJobStatus']
    while _status=='InProgress':
        time.sleep(60)
        _resp   = sm_client.describe_training_job(TrainingJobName=job_name)
        _status = _resp['TrainingJobStatus']
        if _status == 'InProgress':
            print('{} job status: {}'.format(job_name, _status))
    print('DONE. Status for {} is {}\n'.format(job_name, _status))

In [ ]:
# wait for the jobs to finish
for j in training_jobs:
    wait_for_training_job_to_complete(j)

## Import models into hosting
멀티 모델 엔드포인트의 가장 큰 차이점은 모델 엔티티(Model entity)를 작성할 때 컨테이너의 `MultiModel`은 엔드포인트에서 호출할 수 있는 모델 아티팩트가 있는 S3 접두부(prefix)입니다. 나머지 S3 경로는 실제로 모델을 호출할 때 지정됩니다. 슬래시로 위치를 닫아야 하는 점을 기억해 주세요.

컨테이너의 `Mode`는 컨테이너가 여러 모델을 호스팅함을 나타내기 위해 `MultiModel`로 지정됩니다.

### Deploy model artifacts to be found by the endpoint
상술한 바와 같이, 멀티 모델 엔드 포인트는 S3의 특정 위치에서 모델 아티팩트를 찾도록 구성됩니다. 학습된 각 모델에 대해 모델 아티팩트를 해당 위치에 복사합니다.

이 예에서는 모든 모델들을 단일 폴더에 저장합니다. 멀티 모델 엔드 포인트의 구현은 임의의 폴더 구조를 허용할 만큼 유연합니다. 예를 들어 일련의 하우징 모델의 경우 각 지역마다 최상위 폴더가 있을 수 있으며 모델 아티팩트는 해당 지역 폴더로 복사됩니다. 이러한 모델을 호출할 때 참조되는 대상 모델에는 폴더 경로가 포함됩니다. 예를 들어 `northeast/Boston_MA.tar.gz`입니다.

In [ ]:
import re
def parse_model_artifacts(model_data_url):
    # extract the s3 key from the full url to the model artifacts
    _s3_key = model_data_url.split('s3://{}/'.format(BUCKET))[1]
    # get the part of the key that identifies the model within the model artifacts folder
    _model_name_plus = _s3_key[_s3_key.find('model_artifacts') + len('model_artifacts') + 1:]
    # finally, get the unique model name (e.g., "NewYork_NY")
    _model_name = re.findall('^(.*?)/', _model_name_plus)[0]
    return _s3_key, _model_name 

In [ ]:
# make a copy of the model artifacts from the original output of the training job to the place in
# s3 where the multi model endpoint will dynamically load individual models
def deploy_artifacts_to_mme(job_name):
    _resp = sm_client.describe_training_job(TrainingJobName=job_name)
    _source_s3_key, _model_name = parse_model_artifacts(_resp['ModelArtifacts']['S3ModelArtifacts'])
    _copy_source = {'Bucket': BUCKET, 'Key': _source_s3_key}
    _key = '{}/{}/{}.tar.gz'.format(DATA_PREFIX, MULTI_MODEL_ARTIFACTS, _model_name)
    
    print('Copying {} model\n   from: {}\n     to: {}...'.format(_model_name, _source_s3_key, _key))
    s3_client.copy_object(Bucket=BUCKET, CopySource=_copy_source, Key=_key)
    return _key

*의도적으로 첫 번째 모델을 복사하지 않는다는 점을 유의해 주세요.*. 첫 번째 모델은 향후 실습 과정에서 복사하여 이미 실행 중인 엔드포인트에 새 모델을 동적으로 추가하는 방법을 보여주기 위함입니다.

In [ ]:
# First, clear out old versions of the model artifacts from previous runs of this notebook
s3 = boto3.resource('s3')
s3_bucket = s3.Bucket(BUCKET)
full_input_prefix = '{}/multi_model_artifacts'.format(DATA_PREFIX)
print('Removing old model artifacts from {}'.format(full_input_prefix))
filter_resp = s3_bucket.objects.filter(Prefix=full_input_prefix + '/').delete()

In [ ]:
# copy every model except the first one
for job in training_jobs[1:]:
    deploy_artifacts_to_mme(job)

### Create the Amazon SageMaker model entity
`boto3`을 사용하여 모델 엔터티를 만듭니다. 단일 모델을 설명하는 대신 멀티 모델 시맨틱(semantics)의 사용을 나타내며 모든 특정 모델 아티팩트의 소스 위치를 식별합니다.

In [ ]:
# When using multi-model endpoints with the Scikit Learn container, we need to provide an entry point for
# inference that will at least load the saved model. This function uploads a model artifact containing such a
# script. This tar.gz file will be fed to the SageMaker multi-model creation and pointed to by the 
# SAGEMAKER_SUBMIT_DIRECTORY environment variable.

def upload_inference_code(script_file_name, prefix):
    _tmp_folder = 'inference-code'
    if not os.path.exists(_tmp_folder):
        os.makedirs(_tmp_folder)
    !tar -czvf $_tmp_folder/$USER_CODE_ARTIFACTS $script_file_name > /dev/null
    _loc = sagemaker_session.upload_data(_tmp_folder, 
                                         key_prefix='{}/{}'.format(prefix, _tmp_folder))
    return _loc + '/' + USER_CODE_ARTIFACTS

In [ ]:
def create_multi_model_entity(multi_model_name, role):
    # establish the place in S3 from which the endpoint will pull individual models
    _model_url  = 's3://{}/{}/{}/'.format(BUCKET, DATA_PREFIX, MULTI_MODEL_ARTIFACTS)
    _container = {
        'Image':        MULTI_MODEL_SKLEARN_IMAGE,
        'ModelDataUrl': _model_url,
        'Mode':         'MultiModel',
        'Environment': {
            'SAGEMAKER_PROGRAM' : SCRIPT_FILENAME,
            'SAGEMAKER_SUBMIT_DIRECTORY' : upload_inference_code(SCRIPT_FILENAME, DATA_PREFIX)
        }
    }
    create_model_response = sm_client.create_model(
        ModelName = multi_model_name,
        ExecutionRoleArn = role,
        Containers = [_container])
    
    return _model_url

In [ ]:
multi_model_name = '{}-{}'.format(HOUSING_MODEL_NAME, strftime('%Y-%m-%d-%H-%M-%S', gmtime()))
model_url = create_multi_model_entity(multi_model_name, role)
print('Multi model name: {}'.format(multi_model_name))

In [ ]:
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls --human-readable --summarize $model_url

### Create the multi-model endpoint
멀티 모델 엔드포인트에 대한 SageMaker 엔드포인트 설정(config)에는 특별한 것이 없습니다. 예상 예측 워크로드에 적합한 인스턴스 유형과 인스턴스 수를 고려해야 합니다. 개별 모델의 수와 크기에 따라 메모리 요구 사항이 변동합니다.

엔드포인트 설정이 완료되면 엔드포인트 생성(creation)은 간단합니다.

In [ ]:
endpoint_config_name = multi_model_name
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': ENDPOINT_INSTANCE_TYPE,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName'   : multi_model_name,
        'VariantName' : 'AllTraffic'}])

endpoint_name = multi_model_name
print('Endpoint name: ' + endpoint_name)

In [ ]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

In [ ]:
print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

## Exercise the multi-model endpoint

### Invoke multiple individual models hosted behind a single endpoint

여기서 여러분은 특정 위치 기반 주택 모델을 무작위로 선택하는 것을 반복합니다. 주어진 모델의 첫번째 호출에 대해 지불된 콜드 스타트(cold start) 비용이 과금된다는 점을 알아 두세요. 동일한 모델의 후속 호출은 이미 메모리에 로드된 모델을 활용합니다.

In [ ]:
def predict_one_house_value(features, model_name):
    print('Using model {} to predict price of this house: {}'.format(full_model_name,
                                                                     features))

    _float_features = [float(i) for i in features]
    _body = ','.join(map(str, _float_features)) + '\n'
    
    _start_time = time.time()

    _response = runtime_sm_client.invoke_endpoint(
                        EndpointName=endpoint_name,
                        ContentType='text/csv',
                        TargetModel=full_model_name,
                        Body=_body)
    _predicted_value = json.loads(_response['Body'].read())[0]

    _duration = time.time() - _start_time
    
    print('${:,.2f}, took {:,d} ms\n'.format(_predicted_value, int(_duration * 1000)))

In [ ]:
# iterate through invocations with random inputs against a random model showing results and latency
for i in range(10):
    model_name = LOCATIONS[np.random.randint(1, len(LOCATIONS[:PARALLEL_TRAINING_JOBS]))]
    full_model_name = '{}.tar.gz'.format(model_name)
    predict_one_house_value(gen_random_house()[1:], full_model_name)

### Dynamically deploy another model

여기서 신규 모델의 동적 로딩의 힘을 볼 수 있습니다. 이전에 모델을 배포할 때 의도적으로 첫 번째 모델을 복사하지 않았습니다. 이제 추가 모델을 배포하고 다중 모델 엔드 포인트를 통해 즉시 모델을 호출할 수 있습니다. 이전 모델과 마찬가지로 엔드포인트가 모델을 다운로드하고 메모리에 로드하는 데 시간이 걸리므로 새 모델을 처음 호출하는 데 시간이 약간 더 걸린다는 점을 명심해 주세요.

In [ ]:
# add another model to the endpoint and exercise it
deploy_artifacts_to_mme(training_jobs[0])

### Invoke the newly deployed model
엔드포인트 업데이트 또는 재시작 없이 새로 배포된 모델들로 호출을 수행해 보세요.

In [ ]:
print('Here are the models that the endpoint has at its disposal:')
!aws s3 ls $model_url

In [ ]:
model_name = LOCATIONS[0]
full_model_name = '{}.tar.gz'.format(model_name)
for i in range(5):
    features = gen_random_house()
    predict_one_house_value(gen_random_house()[1:], full_model_name)

### Updating a model

모델을 업데이트하려면 위와 동일한 방법으로 새 모델로 추가하세요. 예를 들어,`NewYork_NY.tar.gz` 모델을 재학습하고 호출을 시작하려는 경우 업데이트된 모델 아티팩트를 S3 접두어(prefix) 뒤에 `NewYork_NY_v2.tar.gz`와 같은 새로운 이름으로 업로드한 다음 `NewYork_NY.tar.gz` 대신`NewYork_NY_v2.tar.gz`를 호출하도록 `TargetModel` 필드를 변경하세요. 모델의 이전 버전이 여전히 컨테이너 또는 엔드포인트 인스턴스의 스토리지 볼륨에 로드될 수 있으므로 Amazon S3에서 모델 아티팩트를 덮어 쓰지 않으려고 합니다. 그러면 새 모델 호출 시 이전 버전의 모델을 호출할 수 있습니다.

또는, 엔드포인트를 중지하고 새로운 모델 셋을 재배포할 수 있습니다.

## Clean up
더 이상 사용하지 않는 엔드포인트에 대한 요금이 청구되지 않도록 리소스를 정리합니다.

In [ ]:
# shut down the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
# and the endpoint config
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)

In [ ]:
# delete model too
sm_client.delete_model(ModelName=multi_model_name)